In [1]:
from azure.identity import InteractiveBrowserCredential
from msgraph.core import GraphClient

In [2]:
client_id = 'd5f0b309-f360-482d-8be2-dbd2e6aab632'

In [3]:
browser_credential = InteractiveBrowserCredential(client_id=client_id)

In [4]:
client = GraphClient(credential=browser_credential)

In [ ]:
# result = client.get('/me')
# print(result.json())

In [2]:
import requests
import msal
import atexit
import os.path
import urllib.parse

TENANT_ID = 'consumers'
CLIENT_ID = 'd5f0b309-f360-482d-8be2-dbd2e6aab632'

AUTHORITY = 'https://login.microsoftonline.com/' + TENANT_ID
ENDPOINT = 'https://graph.microsoft.com/v1.0'

SCOPES = [
    'Files.ReadWrite.All',
    'Sites.ReadWrite.All',
    'User.Read',
    'User.ReadBasic.All'
]

cache = msal.SerializableTokenCache()

if os.path.exists('token_cache.bin'):
    cache.deserialize(open('token_cache.bin', 'r').read())

atexit.register(lambda: open('token_cache.bin', 'w').write(cache.serialize()) if cache.has_state_changed else None)

app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY, token_cache=cache)

accounts = app.get_accounts()
result = None
if len(accounts) > 0:
    result = app.acquire_token_silent(SCOPES, account=accounts[0])

if result is None:
    flow = app.initiate_device_flow(scopes=SCOPES)
    if 'user_code' not in flow:
        print(flow)
        raise Exception('Failed to create device flow')

    print(flow['message'])

    result = app.acquire_token_by_device_flow(flow)

if 'access_token' in result:
    headers={'Authorization': 'Bearer ' + result['access_token']}
    params={'allowexternal': 'true'}

    folder = 'SaMo_Grundstücksverwaltungs_GbR'      # path in your OneDrive where file lives
    filename = 'testmappe.xlsx'      # the file you want to download

    # get the drive & file ID
    file_path = f'{folder}/{filename}'
    file_url = urllib.parse.quote(file_path)
#     result = requests.get(f'{ENDPOINT}/me/drive/root:/{file_url}', headers=headers)
    result = requests.get(f'{ENDPOINT}/me/drives', headers=headers, params=params)
    print(result.content)
    result.raise_for_status()
    json = result.json()
    drive_id = json['parentReference']['driveId']
    file_id = json['id']

    # download the file
    result = requests.get(f'{ENDPOINT}/drives/{drive_id}/items/{file_id}/content', headers=headers)
    open('testmappe.xlsx', 'wb').write(result.content)

else:
    raise Exception('no access token in result')

{'error': 'invalid_request', 'error_description': "AADSTS9002332: Application 'd5f0b309-f360-482d-8be2-dbd2e6aab632'(AccChecker) is configured for use by Azure Active Directory users only. Please do not use the /consumers endpoint to serve this request.\r\nTrace ID: a8e6eaef-3166-4d21-a810-08ea4d60620b\r\nCorrelation ID: 5fd9941c-52b4-42f5-a3eb-d19f0f2b9f6b\r\nTimestamp: 2022-01-18 19:38:25Z", 'error_codes': [9002332], 'timestamp': '2022-01-18 19:38:25Z', 'trace_id': 'a8e6eaef-3166-4d21-a810-08ea4d60620b', 'correlation_id': '5fd9941c-52b4-42f5-a3eb-d19f0f2b9f6b', 'interval': 5, 'expires_in': 1800, 'expires_at': 1642536505.038568, '_correlation_id': '5fd9941c-52b4-42f5-a3eb-d19f0f2b9f6b'}


Exception: Failed to create device flow

In [20]:
di = {"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#drives","value":[{"createdDateTime":"2020-06-06T23:35:05Z","description":"","id":"b!c8HSvzsl_0GG1UpekvMeGktCFpFl9IdFheZBDshkRgaxFsZaNYYBSLnPuizDMLDZ","lastModifiedDateTime":"2020-06-06T23:35:05Z","name":"OneDrive","webUrl":"https://stege387-my.sharepoint.com/personal/admin_stege387_onmicrosoft_com/Documents","driveType":"business","createdBy":{"user":{"displayName":"System Account"}},"lastModifiedBy":{"user":{"displayName":"System Account"}},"owner":{"user":{"email":"moritzgeiger@hotmail.de","id":"9e98d297-333a-4006-98c3-90a052598e2a","displayName":"moritz geiger"}},"quota":{"deleted":0,"remaining":1099509994480,"state":"normal","total":1099511627776,"used":1633296}}]}
di    
           
           
           

{'@odata.context': 'https://graph.microsoft.com/v1.0/$metadata#drives',
 'value': [{'createdDateTime': '2020-06-06T23:35:05Z',
   'description': '',
   'id': 'b!c8HSvzsl_0GG1UpekvMeGktCFpFl9IdFheZBDshkRgaxFsZaNYYBSLnPuizDMLDZ',
   'lastModifiedDateTime': '2020-06-06T23:35:05Z',
   'name': 'OneDrive',
   'webUrl': 'https://stege387-my.sharepoint.com/personal/admin_stege387_onmicrosoft_com/Documents',
   'driveType': 'business',
   'createdBy': {'user': {'displayName': 'System Account'}},
   'lastModifiedBy': {'user': {'displayName': 'System Account'}},
   'owner': {'user': {'email': 'moritzgeiger@hotmail.de',
     'id': '9e98d297-333a-4006-98c3-90a052598e2a',
     'displayName': 'moritz geiger'}},
   'quota': {'deleted': 0,
    'remaining': 1099509994480,
    'state': 'normal',
    'total': 1099511627776,
    'used': 1633296}}]}